In [1]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import numpy as np
import matplotlib.pyplot as plt
import wandb

In [2]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import numpy as np
import matplotlib.pyplot as plt
import wandb

In [3]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import numpy as np
import matplotlib.pyplot as plt
import wandb

In [4]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout
import numpy as np
import matplotlib.pyplot as plt

In [5]:
import wandb
wandb.login(key="4d264354a256827a6fb3f5fc1957ba0e0c4b1830")

In [6]:
!pip install wandb --quiet

In [7]:
import wandb
wandb.login(key="4d264354a256827a6fb3f5fc1957ba0e0c4b1830")

True

In [8]:
run = wandb.init(
    name = "ltsm-baseline-moulya", ## Wandb creates random run names if you skip this field
    #reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # id = "1y35a1ce" ,### Insert specific run id here if you want to resume a previous run
    # resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "idl-finance" ### Project should be created in your wandb account 
    # config = config ### Wandb Config for your run
)

In [9]:
from wandb.keras import WandbCallback
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test),callbacks=[WandbCallback(monitor='val_loss', save_model=True)])

In [10]:
stock_prediction_df = pd.read_csv('Code/baseline-implementation/stock_prediction.csv')
stock_prediction_df.drop(columns=['Unnamed: 0'],inplace=True)
stock_prediction_df.head()

         Date  sentiment  PriceDiff
0  2016-01-05   0.400000  -0.459333
1  2016-01-06   0.200000  -0.971214
2  2016-01-07   0.111111   0.116552
3  2016-01-08   0.400000   0.358770
4  2016-01-11   0.400000   0.326792

In [11]:
# Create lagged features for the price_diff values
for i in range(1, 8):
    stock_prediction_df[f'PriceDiff_{i}'] = stock_prediction_df['PriceDiff'].shift(i)

# Remove missing values
stock_prediction_df.dropna(inplace=True)
stock_prediction_df.head()

          Date  sentiment  PriceDiff  PriceDiff_1  PriceDiff_2  PriceDiff_3  \
7   2016-01-14       0.20  -0.546173     0.486757    -0.587307     0.326792   
8   2016-01-15       0.75  -0.107401    -0.546173     0.486757    -0.587307   
9   2016-01-19       0.50   0.029711    -0.107401    -0.546173     0.486757   
10  2016-01-20       0.80  -0.111979     0.029711    -0.107401    -0.546173   
11  2016-01-21       0.20   1.170031    -0.111979     0.029711    -0.107401   

    PriceDiff_4  PriceDiff_5  PriceDiff_6  PriceDiff_7  
7      0.358770     0.116552    -0.971214    -0.459333  
8      0.326792     0.358770     0.116552    -0.971214  
9     -0.587307     0.326792     0.358770     0.116552  
10     0.486757    -0.587307     0.326792     0.358770  
11    -0.546173     0.486757    -0.587307     0.326792  

In [12]:
stock_X = stock_prediction_df[['sentiment', 'PriceDiff_1', 'PriceDiff_2', 'PriceDiff_3', 'PriceDiff_4', 'PriceDiff_5', 'PriceDiff_6', 'PriceDiff_7']]
stock_Y = stock_prediction_df.PriceDiff
X_train = stock_X.loc[:"2016-08-31"]
y_train = stock_Y.loc[:"2016-08-31"]
X_test = stock_X.loc["2016-09-01":]
y_test = stock_Y.loc["2016-09-01":]

In [13]:
# Build the model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.1))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=50))
model.add(Dropout(0.1))
model.add(Dense(units=1))
model.summary()

In [14]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [15]:
from wandb.keras import WandbCallback
model.fit(X_train, y_train, epochs=200, batch_size=32, validation_data=(X_test, y_test),callbacks=[WandbCallback(monitor='val_loss', save_model=True)])

In [16]:
predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)
predicted_price_diff = np.concatenate((predictions_train,predictions_test),axis=0)

In [17]:
stock_prediction_df['Predicted Price Diff'] = predicted_price_diff
stock_prediction_df.head()

          Date  sentiment  PriceDiff  PriceDiff_1  PriceDiff_2  PriceDiff_3  \
7   2016-01-14       0.20  -0.546173     0.486757    -0.587307     0.326792   
8   2016-01-15       0.75  -0.107401    -0.546173     0.486757    -0.587307   
9   2016-01-19       0.50   0.029711    -0.107401    -0.546173     0.486757   
10  2016-01-20       0.80  -0.111979     0.029711    -0.107401    -0.546173   
11  2016-01-21       0.20   1.170031    -0.111979     0.029711    -0.107401   

    PriceDiff_4  PriceDiff_5  PriceDiff_6  PriceDiff_7  Predicted Price Diff  
7      0.358770     0.116552    -0.971214    -0.459333             -0.265381  
8      0.326792     0.358770     0.116552    -0.971214             -0.274767  
9     -0.587307     0.326792     0.358770     0.116552              0.056106  
10     0.486757    -0.587307     0.326792     0.358770              0.058799  
11    -0.546173     0.486757    -0.587307     0.326792              0.112918  

In [18]:
fig, ax = plt.subplots(figsize=(10, 6))
stock_prediction_df.plot(y='PriceDiff',color='green',ax=ax)
stock_prediction_df.loc['2016-08-01':, 'Predicted Price Diff'].plot(ax=ax, color='red', label='Predicted Price Diff')
# stock_prediction_df.plot(y='Predicted Price Diff',color='red',ax=ax)

<AxesSubplot: >

In [19]:
run = wandb.init(
    name = "ltsm-baseline-moulya-1000 epochs", ## Wandb creates random run names if you skip this field
    #reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # id = "1y35a1ce" ,### Insert specific run id here if you want to resume a previous run
    # resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "idl-finance" ### Project should be created in your wandb account 
    # config = config ### Wandb Config for your run
)

In [20]:
from wandb.keras import WandbCallback
model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test),callbacks=[WandbCallback(monitor='val_loss', save_model=True)])

In [21]:
predictions_train = model.predict(X_train)
predictions_test = model.predict(X_test)
predicted_price_diff = np.concatenate((predictions_train,predictions_test),axis=0)

In [22]:
stock_prediction_df['Predicted Price Diff'] = predicted_price_diff
stock_prediction_df.head()

          Date  sentiment  PriceDiff  PriceDiff_1  PriceDiff_2  PriceDiff_3  \
7   2016-01-14       0.20  -0.546173     0.486757    -0.587307     0.326792   
8   2016-01-15       0.75  -0.107401    -0.546173     0.486757    -0.587307   
9   2016-01-19       0.50   0.029711    -0.107401    -0.546173     0.486757   
10  2016-01-20       0.80  -0.111979     0.029711    -0.107401    -0.546173   
11  2016-01-21       0.20   1.170031    -0.111979     0.029711    -0.107401   

    PriceDiff_4  PriceDiff_5  PriceDiff_6  PriceDiff_7  Predicted Price Diff  
7      0.358770     0.116552    -0.971214    -0.459333             -0.519101  
8      0.326792     0.358770     0.116552    -0.971214             -0.103464  
9     -0.587307     0.326792     0.358770     0.116552              0.020325  
10     0.486757    -0.587307     0.326792     0.358770             -0.119176  
11    -0.546173     0.486757    -0.587307     0.326792              1.148149  

In [23]:
fig, ax = plt.subplots(figsize=(10, 6))
stock_prediction_df.plot(y='PriceDiff',color='green',ax=ax)
stock_prediction_df.loc['2016-08-01':, 'Predicted Price Diff'].plot(ax=ax, color='red', label='Predicted Price Diff')
# stock_prediction_df.plot(y='Predicted Price Diff',color='red',ax=ax)

<AxesSubplot: >

In [24]:
model = Sequential()
model.add(Bidirectional(LSTM(units=64, return_sequences=True, input_shape=(X_train.shape[1], 1))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=64)))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.summary()

In [25]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
import numpy as np
import matplotlib.pyplot as plt

In [26]:
model = Sequential()
model.add(Bidirectional(LSTM(units=64, return_sequences=True, input_shape=(X_train.shape[1], 1))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=64)))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.summary()

In [27]:
stock_prediction_df = pd.read_csv('Code/baseline-implementation/stock_prediction.csv')
stock_prediction_df.drop(columns=['Unnamed: 0'],inplace=True)
stock_prediction_df.head()

         Date  sentiment  PriceDiff
0  2016-01-05   0.400000  -0.459333
1  2016-01-06   0.200000  -0.971214
2  2016-01-07   0.111111   0.116552
3  2016-01-08   0.400000   0.358770
4  2016-01-11   0.400000   0.326792

In [28]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
import numpy as np
import matplotlib.pyplot as plt

In [29]:
!pip install wandb --quiet

In [30]:
import wandb
wandb.login(key="4d264354a256827a6fb3f5fc1957ba0e0c4b1830")

True

In [31]:
stock_prediction_df = pd.read_csv('Code/baseline-implementation/stock_prediction.csv')
stock_prediction_df.drop(columns=['Unnamed: 0'],inplace=True)
stock_prediction_df.head()

         Date  sentiment  PriceDiff
0  2016-01-05   0.400000  -0.459333
1  2016-01-06   0.200000  -0.971214
2  2016-01-07   0.111111   0.116552
3  2016-01-08   0.400000   0.358770
4  2016-01-11   0.400000   0.326792

In [32]:
# Create lagged features for the price_diff values
for i in range(1, 8):
    stock_prediction_df[f'PriceDiff_{i}'] = stock_prediction_df['PriceDiff'].shift(i)

# Remove missing values
stock_prediction_df.dropna(inplace=True)
stock_prediction_df.head()

          Date  sentiment  PriceDiff  PriceDiff_1  PriceDiff_2  PriceDiff_3  \
7   2016-01-14       0.20  -0.546173     0.486757    -0.587307     0.326792   
8   2016-01-15       0.75  -0.107401    -0.546173     0.486757    -0.587307   
9   2016-01-19       0.50   0.029711    -0.107401    -0.546173     0.486757   
10  2016-01-20       0.80  -0.111979     0.029711    -0.107401    -0.546173   
11  2016-01-21       0.20   1.170031    -0.111979     0.029711    -0.107401   

    PriceDiff_4  PriceDiff_5  PriceDiff_6  PriceDiff_7  
7      0.358770     0.116552    -0.971214    -0.459333  
8      0.326792     0.358770     0.116552    -0.971214  
9     -0.587307     0.326792     0.358770     0.116552  
10     0.486757    -0.587307     0.326792     0.358770  
11    -0.546173     0.486757    -0.587307     0.326792  

In [33]:
stock_X = stock_prediction_df[['sentiment', 'PriceDiff_1', 'PriceDiff_2', 'PriceDiff_3', 'PriceDiff_4', 'PriceDiff_5', 'PriceDiff_6', 'PriceDiff_7']]
stock_Y = stock_prediction_df.PriceDiff
X_train = stock_X.loc[:"2016-08-31"]
y_train = stock_Y.loc[:"2016-08-31"]
X_test = stock_X.loc["2016-09-01":]
y_test = stock_Y.loc["2016-09-01":]

In [34]:
# Build the model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.1))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=50))
model.add(Dropout(0.1))
model.add(Dense(units=1))
model.summary()

In [35]:
model = Sequential()
model.add(Bidirectional(LSTM(units=64, return_sequences=True, input_shape=(X_train.shape[1], 1))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=64)))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.summary()

In [36]:
model = Sequential()
model.add(Bidirectional(LSTM(units=64, return_sequences=True, input_shape=(X_train.shape[1], 1))))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=64, return_sequences=True)))
model.add(Dropout(0.2))
model.add(Bidirectional(LSTM(units=64)))
model.add(Dropout(0.2))
model.add(Dense(units=1))
# model.summary()

In [37]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [38]:
run = wandb.init(
    name = "bi-ltsm-baseline-moulya-1000 epochs", ## Wandb creates random run names if you skip this field
    #reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # id = "1y35a1ce" ,### Insert specific run id here if you want to resume a previous run
    # resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "idl-finance" ### Project should be created in your wandb account 
    # config = config ### Wandb Config for your run
)

epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
loss,█▇▇▇▆▅▅▄▄▃▄▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▂▁▁▁▁▁▁▁
val_loss,▁▁▁▁▁▂▃▃▅▇▄▆█▅▆▆▇▇▇▆▅▆▆▅▇▆▇▆▆▅▆▆▄▅▅▄▄▅▄▅
best_epoch,108
best_val_loss,0.07667
epoch,999
loss,0.00336
val_loss,0.11711


In [39]:
from wandb.keras import WandbCallback
model.fit(X_train, y_train, epochs=1000, batch_size=32, validation_data=(X_test, y_test),callbacks=[WandbCallback(monitor='val_loss', save_model=True)])

In [40]:
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout, Bidirectional
import numpy as np
import matplotlib.pyplot as plt

In [41]:
!pip install wandb --quiet

In [42]:
import wandb
wandb.login(key="4d264354a256827a6fb3f5fc1957ba0e0c4b1830")

True

In [43]:
!pwd

In [44]:
zoom_train_df = pd.read_csv('Code/train-model/data/zoom/zoom-train-date-sentiment-pricediff.csv')
zoom_train_df.head()

         Date  sentiment_score     Close
0  2020-01-06           0.9001  1.580002
1  2020-01-07           0.2787  0.650002
2  2020-01-08           0.0000  0.070000
3  2020-01-15           0.0000 -0.830002
4  2020-01-21           0.3400 -1.190002

In [45]:
zoom_train_df = pd.read_csv('Code/train-model/data/zoom/zoom-train-date-sentiment-pricediff.csv')
zoom_train_df.head()

         Date  sentiment_score     Close
0  2020-01-06           0.9001  1.580002
1  2020-01-07           0.2787  0.650002
2  2020-01-08           0.0000  0.070000
3  2020-01-15           0.0000 -0.830002
4  2020-01-21           0.3400 -1.190002

In [46]:
zoom_test_df = pd.read_csv('Code/train-model/data/zoom/zoom-tweets-test-date-sentiment-pricediff.csv')
zoom_test_df.head()

         Date  sentiment_score     Close
0  2020-01-02          -0.0516 -1.440002
1  2020-01-03           0.0057  3.040001
2  2020-01-06           0.0000  1.580002
3  2020-01-07           0.4404  0.650002
4  2020-01-08           0.2537  0.070000

In [47]:
# Create lagged features for the price_diff values
for i in range(1, 8):
    zoom_train_df[f'PriceDiff_{i}'] = zoom_train_df['Close'].shift(i)

# Remove missing values
zoom_train_df.dropna(inplace=True)
zoom_train_df.head()

          Date  sentiment_score      Close  PriceDiff_1  PriceDiff_2  \
7   2020-01-31          0.52670  11.360001     1.760002     2.070000   
8   2020-02-03          0.31105  -1.870003    11.360001     1.760002   
9   2020-02-10          0.00000  -0.989998    -1.870003    11.360001   
10  2020-02-28          0.00000   8.110001    -0.989998    -1.870003   
11  2020-03-02          0.00000   4.360001     8.110001    -0.989998   

    PriceDiff_3  PriceDiff_4  PriceDiff_5  PriceDiff_6  PriceDiff_7  
7     -1.190002    -0.830002     0.070000     0.650002     1.580002  
8      2.070000    -1.190002    -0.830002     0.070000     0.650002  
9      1.760002     2.070000    -1.190002    -0.830002     0.070000  
10    11.360001     1.760002     2.070000    -1.190002    -0.830002  
11    -1.870003    11.360001     1.760002     2.070000    -1.190002  

In [48]:
# Create lagged features for the price_diff values
for i in range(1, 8):
    zoom_test_df[f'PriceDiff_{i}'] = zoom_test_df['Close'].shift(i)

# Remove missing values
zoom_test_df.dropna(inplace=True)
zoom_test_df.head()

          Date  sentiment_score     Close  PriceDiff_1  PriceDiff_2  \
7   2020-01-13         0.028800 -0.869995     0.940002     0.469994   
8   2020-01-14        -0.002720  3.779999    -0.869995     0.940002   
9   2020-01-15         0.051825 -0.830002     3.779999    -0.869995   
10  2020-01-16         0.466225 -0.550003    -0.830002     3.779999   
11  2020-01-17         0.128560  1.170006    -0.550003    -0.830002   

    PriceDiff_3  PriceDiff_4  PriceDiff_5  PriceDiff_6  PriceDiff_7  
7      0.070000     0.650002     1.580002     3.040001    -1.440002  
8      0.469994     0.070000     0.650002     1.580002     3.040001  
9      0.940002     0.469994     0.070000     0.650002     1.580002  
10    -0.869995     0.940002     0.469994     0.070000     0.650002  
11     3.779999    -0.869995     0.940002     0.469994     0.070000  

In [49]:
X_train = zoom_train_df[['sentiment_score', 'PriceDiff_1', 'PriceDiff_2', 'PriceDiff_3', 'PriceDiff_4', 'PriceDiff_5', 'PriceDiff_6', 'PriceDiff_7']]
y_train = zoom_train_df.Close
X_test = zoom_test_df[['sentiment_score', 'PriceDiff_1', 'PriceDiff_2', 'PriceDiff_3', 'PriceDiff_4', 'PriceDiff_5', 'PriceDiff_6', 'PriceDiff_7']]
y_test = zoom_test_df.Close

In [50]:
# Build the model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(Dropout(0.1))
model.add(LSTM(units=50, return_sequences=True))
model.add(Dropout(0.1))
model.add(LSTM(units=50))
model.add(Dropout(0.1))
model.add(Dense(units=1))
model.summary()

In [51]:
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')

In [52]:
run = wandb.init(
    name = "ltsm-zoom-moulya-200-epochs", ## Wandb creates random run names if you skip this field
    #reinit = True, ### Allows reinitalizing runs when you re-run this cell
    # id = "1y35a1ce" ,### Insert specific run id here if you want to resume a previous run
    # resume = "must", ### You need this to resume previous runs, but comment out reinit = True when using this
    project = "idl-finance" ### Project should be created in your wandb account 
    # config = config ### Wandb Config for your run
)